In [2]:
import numpy as np
ctrl_bounds = [(-1,1)]* 10
state_bounds = []
feed_bounds = state_bounds + ctrl_bounds
feed_bounds

[(-1, 1),
 (-1, 1),
 (-1, 1),
 (-1, 1),
 (-1, 1),
 (-1, 1),
 (-1, 1),
 (-1, 1),
 (-1, 1),
 (-1, 1)]

In [8]:
2 * upper_input_bound

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [7]:
lower_input_bound

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [3]:
z = 10
lower_input_bound = [x[0] for x in feed_bounds]
upper_input_bound = [x[1] for x in feed_bounds]
inputs_set = np.random.uniform(low=2*lower_input_bound, high=2*upper_input_bound, size=(5, 2*z))

In [5]:
inputs_set.shape

(5, 20)

In [6]:
import random

demos = [[(0, 1), (0, 1), (0, 1)], [(2, 1), (2, 1), (2, 1)], [(4, 3), (4, 3), (4, 3)], [(3, 3), (3, 3), (3, 3)]]

In [7]:
demos_shuffled = demos.copy()
random.shuffle(demos_shuffled)

In [11]:
for i in range(4):
    print(i)

0
1
2
3


In [1]:
import random
#import mdp_utils
#import mdp_worlds
#import bayesian_irl
#from mdp import FeatureMDP
import copy
from scipy.stats import norm
import numpy as np
import math
import argparse

import sys
import os
import time
import yaml
import numpy as np
import random


# Get the current working directory of the notebook
current_directory = os.getcwd()

# Get the parent directory
parent_directory = os.path.dirname(current_directory)

# Add the parent directory to the system path
sys.path.append(parent_directory)

# Load YAML file from the parent directory (assuming it's in a 'configs' folder within the parent directory)
yaml_file_path = os.path.join(parent_directory, 'configs', 'gridworld_config.yaml')

# Check if the YAML file exists and load it
if os.path.exists(yaml_file_path):
    with open(yaml_file_path, 'r') as file:
        config = yaml.safe_load(file)
else:
    raise FileNotFoundError(f"YAML file not found at: {yaml_file_path}")


from env import gridworld_env, gridworld_env2
from agent.q_learning_agent import ValueIteration
from data_generation.generate_data import GridWorldMDPDataGenerator
from reward_learning.birl import BIRL
from utils.common_helper import (calculate_percentage_optimal_actions,
                                 compute_policy_loss_avar_bound,
                                 calculate_expected_value_difference)

rseed = 1377
random.seed(rseed)
np.random.seed(rseed)


render_mode = config['env_config']['render_mode']
size = config['env_config']['size']
noise_prob = config['env_config']['noise_prob']
seed = config['env_config']['seed']
gamma = config['env_config']['gamma']
epsilon = float(config['algorithm_config']['epsilon'])

num_steps = config['bayesian_irl_config']['num_steps']
step_stdev = config['bayesian_irl_config']['step_stdev']
beta = config['bayesian_irl_config']['beta']
normalize = config['bayesian_irl_config']['normalize']
adaptive = config['bayesian_irl_config']['adaptive']
burn_frac = config['bayesian_irl_config']['burn_frac']
skip_rate = config['bayesian_irl_config']['skip_rate']



alphas = config['suff_config']['alphas']
delta = config['suff_config']['delta']
optimality_threshold = config['suff_config']['optimality_threshold']
random_normalization = config['suff_config']['random_normalization']
thresholds = config['suff_config']['thresholds']


num_world = config['experiments']['num_world']
num_demonstration = config['experiments']['num_demonstration']

In [ ]:
import random

In [ ]:
demo = [([(0,1), (3,3), (4,3), (5,None)], [(0,1), (3,0), (0,1), (3,0)]), ([(0,1), (3,3), (4,3), (5,None)], [(0,3), (1,3), (2,1), (5,None)])].copy()

random.shuffle(demo)

In [ ]:
demo

In [ ]:
demo[1]

In [ ]:
def compute_reward_for_trajectory(env, trajectory, discount_factor=None):
    """
    Computes the cumulative reward for a given trajectory in the environment. If a discount factor 
    is provided, the function calculates the discounted cumulative reward, where rewards received 
    later in the trajectory are given less weight.

    :param env: The environment (MDP) which provides the reward function. This should have a 
                `compute_reward(state)` method.
    :param trajectory: List of tuples (state, action), where `state` is the current state and 
                       `action` is the action taken in that state. The action is ignored in reward 
                       computation but kept for compatibility with the trajectory format.
    :param discount_factor: (Optional) A float representing the discount factor (gamma) for 
                            future rewards. It should be between 0 and 1. If None, no discounting 
                            is applied, and rewards are summed without any decay.
    :return: The cumulative reward for the trajectory, either discounted or non-discounted.
             If discount_factor is provided, it applies a discount based on the time step of 
             the trajectory.
    """
    cumulative_reward = 0
    discount = 1 if discount_factor is None else discount_factor
    
    for t, (state, action) in enumerate(trajectory):
        if state is None:  # Terminal state reached
            break
        
        # Compute the reward for the current state
        reward = env.compute_reward(state)
        
        # If a discount factor is provided, apply it to the reward
        if discount_factor:
            cumulative_reward += reward * (discount_factor ** t)
        else:
            cumulative_reward += reward

    return cumulative_reward

In [ ]:
traj = [(21, 1), (21, 0), (16, 3), (17, 3), (18, 1), (23, 3), (24, None)]

In [ ]:
color_to_feature_map = {
    "red": [1, 0, 0],
    "blue": [0, 1, 0],
    "black": [0, 0, 1]  # 'black' indicates a terminal state
}

custom_grid_features = [
    ["blue", "red", "blue"],
    ["blue", "blue", "black"]
]

# Initialize environments
# Define your feature weights list
feature_weights_list = [[-1, -0.3, 1]]

# Initialize environments with feature weights
env = gridworld_env2.NoisyLinearRewardFeaturizedGridWorldEnv(gamma=gamma,
    color_to_feature_map=color_to_feature_map,
    grid_features=custom_grid_features,
    custom_feature_weights=feature_weights_list[0]) 

In [ ]:
([(0,1), (3,3), (4,3), (5,None)], [(0,3), (1,3), (2,1), (5,None)]) , ([(0,1), (3,3), (4,3), (5,None)], [(0,1), (3,0), (0,1), (3,0)])

In [ ]:

RIGHT = 3
UP = 0
LEFT = 2
DOWN = 1

traj = [(0,3), (1,3), (2,1), (5,None)]

compute_reward_for_trajectory(env, traj, discount_factor=None)

In [ ]:
traj = [(0,1), (3,3), (4,3), (5,None)]

compute_reward_for_trajectory(env, traj, discount_factor=None)

In [ ]:
traj = [(0,1), (3,0), (0,1), (3,0)]

compute_reward_for_trajectory(env, traj, discount_factor=None)

In [ ]:
import pybullet as p
import pybullet_data
import time
import numpy as np
from scipy.interpolate import CubicSpline

class Trajectory:
    def __init__(self, xi, T):
        """
        Initialize the trajectory object.
        
        Args:
            xi: A 2D array (n x m) where n is the number of waypoints and m is the number of joints.
            T: The total time duration for the trajectory.
        """
        self.T = T
        self.n, self.m = np.shape(xi)
        self.traj = []
        xi = np.asarray(xi)
        timesteps = np.linspace(0, self.T, self.n)
        
        # Use CubicSpline for smoother interpolation
        for idx in range(self.m):
            self.traj.append(CubicSpline(timesteps, xi[:, idx]))
        
    def get_waypoint(self, t):
        """
        Get the waypoint at time t.
        
        Args:
            t: The time at which to sample the trajectory.
            
        Returns:
            A 1D array containing the joint positions at time t.
        """
        if t < 0.0:
            t = 0.0
        if t > self.T:
            t = self.T
        
        waypoint = np.array([0.] * self.m)
        for idx in range(self.m):
            waypoint[idx] = self.traj[idx](t)
        
        return waypoint

# Connect to PyBullet
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load the Panda robot URDF
robot_id = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)

# Set gravity for the simulation
p.setGravity(0, 0, -9.81)

# Number of joints (typically 7 for Panda arm)
num_joints = 7

# Replace this with your actual sequence of joint positions for each joint (each row should have 7 values)
joint_positions_sequence = [
 [-2.75771,0.879738,-0.203636,0.867466,-0.5184,-0.02646,3.04725],
 [-3.0323,1.3127,-0.859796,0.333257,0.234699,1.03275,3.04725],
[-2.75771,0.879738,-0.203636,0.867466,-0.518486,-0.0264612,3.04725],



]


# Create a Trajectory instance for generating waypoints
T = 2.0  # Total duration in seconds
trajectory = Trajectory(joint_positions_sequence, T)

# Set up the camera for a zoomed-in view
cameraDistance = 1.0  # Smaller values for closer zoom
cameraYaw = 50        # Horizontal rotation of the camera
cameraPitch = -30     # Vertical angle of the camera
cameraTargetPosition = [0, 0, 0.5]  # Center point the camera looks at

p.resetDebugVisualizerCamera(cameraDistance, cameraYaw, cameraPitch, cameraTargetPosition)

# Simulation loop to visualize the joint movements
dt = 0.5  # Time step for visualization
current_time = 0.0

while current_time <= T:
    joint_positions = trajectory.get_waypoint(current_time)
    for joint_index in range(num_joints):
        p.setJointMotorControl2(
            bodyIndex=robot_id,
            jointIndex=joint_index,
            controlMode=p.POSITION_CONTROL,
            targetPosition=joint_positions[joint_index]
        )
    # Step the simulation
    p.stepSimulation()
    # Add a delay for visualization
    time.sleep(dt)
    current_time += dt

# Keep the simulation window open until closed by the user
input("Press Enter to exit...")

# Disconnect from PyBullet
#p.disconnect()


In [ ]:
def generate_estop(env, beta, trajs):
    """
    Generates E-stops for random trajectories using the human likelihood model.
    
    :param beta: Sensitivity parameter for human decision-making.
    :param num_trajs: Number of trajectories to generate.
    :return: List of E-stop events (trajectories with stopping point t).
    """
    # Generate random trajectories first

    estop_trajectories = []

    # Iterate over each trajectory
    for trajectory in trajs:
        T = len(trajectory)  # Length of the trajectory
        stop_probs = []
        
        # Calculate stop probabilities for each possible stopping point t
        for t in range(T):
            # Sub-trajectory ξ_0:t (use cumulative rewards up to point t)
            reward_up_to_t = sum(env.compute_reward(s) for s, _ in trajectory[:t+1])
            stop_prob_numerator = np.exp(beta * reward_up_to_t)
            
            # Compute denominator (normalization factor for the entire trajectory)
            stop_prob_denominator = sum(np.exp(beta * sum(env.compute_reward(s) for s, _ in trajectory[:k+1])) for k in range(T))
            
            # Calculate stop probability for stopping at time t
            stop_prob = stop_prob_numerator / stop_prob_denominator
            stop_probs.append(stop_prob)
        
        # Select stopping point t based on the calculated probabilities
        #stop_point = np.random.choice(range(T), p=stop_probs)
        #print(stop_prob)
        stop_point = np.argmax(stop_probs)
        
        # Append the trajectory with its stopping point to the result list
        estop_trajectories.append((trajectory, stop_point))

    return estop_trajectories

In [ ]:
for t in range(3):
    print(t)

In [35]:
import numpy as np

def generate_estop(env, beta, trajs):
    """
    Generates E-stops for random trajectories using the human likelihood model with discounted rewards.

    :param env: The environment that provides rewards for states.
    :param beta: Sensitivity parameter for human decision-making.
    :param trajs: List of trajectories to generate E-stops for.
    :return: List of E-stop events (trajectories with stopping point t).
    """
    estop_trajectories = []
    discount_factor = 1

    # Iterate over each trajectory
    for trajectory in trajs:
        T = len(trajectory)  # Length of the trajectory
        stop_probs = []

        # Calculate stop probabilities for each possible stopping point t
        for t in range(T):
            # Calculate discounted cumulative reward up to point t
            reward_up_to_t = sum(
                (discount_factor ** i) * env.compute_reward(s) for i, (s, _) in enumerate(trajectory[:t+1])
            )
            stop_prob_numerator = np.exp(beta * reward_up_to_t)

            # Compute denominator (normalization factor for the entire trajectory)
            stop_prob_denominator = sum(
                np.exp(beta * sum((discount_factor ** i) * env.compute_reward(s) for i, (s, _) in enumerate(trajectory[:k+1])))
                for k in range(T)
            )

            # Calculate stop probability for stopping at time t
            stop_prob = stop_prob_numerator / stop_prob_denominator
            stop_probs.append(stop_prob)

        # Select stopping point t with the highest stop probability
        stop_point = np.argmax(stop_probs)

        # Append the trajectory with its stopping point to the result list
        estop_trajectories.append((trajectory, stop_point))

    return estop_trajectories


In [78]:
import numpy as np

reward_param = [-1, 0.1, 0.5]
reward_param = np.array(reward_param)/np.linalg.norm(reward_param)
print(reward_param)

[-0.89087081  0.08908708  0.4454354 ]


In [79]:
color_to_feature_map = {
    "red": [1, 0, 0],
    "blue": [0, 1, 0],
    "black": [0, 0, 1]  # 'black' indicates a terminal state
}

custom_grid_features = [
    ["blue", "red", "blue"],
    ["blue", "blue", "black"]
]

# Initialize environments
# Define your feature weights list
feature_weights_list = list(reward_param)
# Initialize environments with feature weights
env = gridworld_env2.NoisyLinearRewardFeaturizedGridWorldEnv(gamma=gamma,
    color_to_feature_map=color_to_feature_map,
    grid_features=custom_grid_features,
    custom_feature_weights=feature_weights_list)

In [81]:
RIGHT = 3
UP = 0
LEFT = 2
DOWN = 1

generate_estop(env, 10, [
    [(0,1), (3,3), (4,0), (1,3), (2,1) ,(5,None)],
    [(0,1), (3,3), (4,0), (1,1), (4,3) ,(5,None)],
    [(0,1), (3,3), (4,1), (4,0), (1,3), (2,1) ,(5,None)],
    [(0,1), (3,3), (4,3), (5,None)],
    [(3,3), (4,0), (1,1), (4,3) ,(5,None)],])

[([(0, 1), (3, 3), (4, 0), (1, 3), (2, 1), (5, None)], 2),
 ([(0, 1), (3, 3), (4, 0), (1, 1), (4, 3), (5, None)], 2),
 ([(0, 1), (3, 3), (4, 1), (4, 0), (1, 3), (2, 1), (5, None)], 3),
 ([(0, 1), (3, 3), (4, 3), (5, None)], 3),
 ([(3, 3), (4, 0), (1, 1), (4, 3), (5, None)], 1)]